In [1]:
#from multiprocessing import Pool
#from functools import partial
import numpy as np
#from numba import jit

In [2]:
#TODO: loss of least square regression and binary logistic regression
'''
    pred() takes GBDT/RF outputs, i.e., the "score", as its inputs, and returns predictions.
    g() is the gradient/1st order derivative, which takes true values "true" and scores as input, and returns gradient.
    h() is the heassian/2nd order derivative, which takes true values "true" and scores as input, and returns hessian.
'''
class leastsquare(object):
    '''Loss class for mse. As for mse, pred function is pred=score.'''
    def pred(self,score):
        return score

    def g(self,true,score):
        return 2 * (score - true)

    def h(self,true,score):
        return 2 * np.ones_like(score)

class logistic(object):
    '''Loss class for log loss. As for log loss, pred function is logistic transformation.'''
    def pred(self,score):
        return 1 / (1 + np.exp(-score))

    def g(self,true,score):
        return self.pred(score) - true

    def h(self,true,score):
        return self.pred(score) * (1 - self.pred(score))

In [3]:
# TODO: class of a node on a tree
from multiprocessing import Pool
import multiprocessing as mp 
mp.set_start_method('fork')

class TreeNode(object):
    '''
    Data structure that are used for storing a node on a tree.
    
    A tree is presented by a set of nested TreeNodes,
    with one TreeNode pointing two child TreeNodes,
    until a tree leaf is reached.
    
    A node on a tree can be either a leaf node or a non-leaf node.
    '''
    
    #TODO
    # def __init__(self, X, y, depth):
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None, depth=None, is_leaf=False):
        # store essential information in every tree node
        self.is_leaf = False if (feature is not None and threshold is not None) else True
        self.feature = feature  # feature index for split
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        self.depth = depth 
        self.is_leaf = is_leaf



# TODO: class of single tree
class Tree(object):
    '''
    Class of a single decision tree in GBDT

    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        max_depth: The maximum depth of the tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf prediction, also known as lambda.
        gamma: The regularization coefficient for number of TreeNode, also know as gamma.
        rf: rf*m is the size of random subset of features, from which we select the best decision rule,
            rf = 0 means we are training a GBDT.
    '''
    
    def __init__(self, n_threads = None, 
                 max_depth = 5, min_sample_split = 5,
                 lamda = 2, gamma = 1, rf = 0.5, ):
        self.n_threads = n_threads
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.rf = rf
        self.int_member = 0
        self.multiproc = True if n_threads is not None and n_threads > 1 else False

    def _leaf_value(self, g, h):
        den = h.sum() + self.lamda
        return -g.sum() / den if den > 0 else 0.0

    def fit(self, train, g, h):
        '''
        train is the training data matrix, and must be numpy array (an n_train x m matrix).
        g and h are gradient and hessian respectively.
        '''
        #TODO
        # Initialize the root node
        root = self.construct_tree(train, g, h)
        # Set the root node to the tree
        self.root = root
        return self

    def predict(self,test):
        '''
        test is the test data matrix, and must be numpy arrays (an n_test x m matrix).
        Return predictions (scores) as an array.
        '''
        #TODO
        result = np.zeros(test.shape[0])
        # Traverse the tree for each test sample
        for i in range(test.shape[0]):
            node = self.root
            while not node.is_leaf:
                if test[i, node.feature] <= node.threshold:
                    node = node.left
                else:
                    node = node.right
            result[i] = node.value
        # Return the predictions
        return result

    def construct_tree(self, X, g, h, depth=0):
        """
        Recursively build one tree in GBDT.
        Stops when:
        1. depth >= self.max_depth
        2. number of samples < self.min_sample_split
        3. best gain <= 0
        """
        # 1. Stopping condition: too deep or too few samples
        n_samples = X.shape[0]
        if depth >= self.max_depth or n_samples < self.min_sample_split:
            leaf_val = self._leaf_value(g, h)
            return TreeNode(value=leaf_val, depth=depth, is_leaf=True)

        # 2. Find best split
        feat, thr, gain = self.find_best_decision_rule(X, g, h)
        if gain is None or gain <= 0:
            leaf_val = self._leaf_value(g, h)
            return TreeNode(value=leaf_val, depth=depth, is_leaf=True)

        # 3. Split data
        mask_left = X[:, feat] <= thr
        mask_right = ~mask_left

        # 4. Ensure no empty or undersized child
        if mask_left.sum() < self.min_sample_split or mask_right.sum() < self.min_sample_split:
            leaf_val = self._leaf_value(g, h)
            return TreeNode(value=leaf_val, depth=depth, is_leaf=True)

        # 5. Recurse
        left  = self.construct_tree(X[mask_left],  g[mask_left],  h[mask_left],  depth+1)
        right = self.construct_tree(X[mask_right], g[mask_right], h[mask_right], depth+1)

        # 6. Return internal node
        return TreeNode(
            feature=feat,
            threshold=thr,
            left=left,
            right=right,
            depth=depth,
            is_leaf=False
        )


    def find_best_decision_rule(self, train, g, h):
        '''
        Return the best decision rule [feature, treshold], i.e., $(p_j, \tau_j)$ on a node j, 
        train is the training data assigned to node j
        g and h are the corresponding 1st and 2nd derivatives for each data point in train
        g and h should be vectors of the same length as the number of data points in train
        
        for each feature, we find the best threshold by find_threshold(),
        a [threshold, best_gain] list is returned for each feature.
        Then we select the feature with the largest best_gain,
        and return the best decision rule [feature, treshold] together with its gain.
        '''
        #TODO
        # Version without multiprocessing.
        # n_features = np.arange(train.shape[1])
        # f_size = int(n_features.size * self.rf)
        # feats = np.random.choice(n_features, size=f_size, replace=False) if self.rf > 0 else n_features
        # best_gain = -np.inf
        # best_feat = None
        # best_thr = None
        # threshold = None
        # for f in feats:
        #     # Find the best threshold and gain for feature f
        #     threshold, gain = self.find_threshold(g, h, train[:, f])
        #     if gain > best_gain:
        #         best_gain = gain
        #         best_feat = f
        #         best_thr = threshold
        # return best_feat, best_thr, best_gain
        n_samples, n_features = train.shape
        best_gain = -np.inf
        best_feature = None
        best_threshold = None
        if self.rf > 0:
            n_sub = max(1, int(self.rf * n_features))
            feature_indices = np.random.choice(n_features, n_sub, replace=False)
        else:
            feature_indices = range(n_features)
        args_list = [(train[:, i], g, h) for i in feature_indices]
        if self.multiproc and self.n_threads > 1:
            with Pool(self.n_threads) as pool:
                results = pool.map(self._threshold_wrapper, args_list)
        else:
            results = map(self._threshold_wrapper, args_list)
        for feat_idx, (thr, gain) in zip(feature_indices, results):
            if gain > best_gain:
                best_gain = gain
                best_feature = feat_idx
                best_threshold = thr

        return best_feature, best_threshold, best_gain
    
    def _threshold_wrapper(self, args):
        # 这时 args = (feature_col, g, h)
        feat_col, g, h = args
        return self.find_threshold(g, h, feat_col)

    def find_threshold(self, g, h, feature_col):
        """
        Given gradients g, hessians h and a single feature column,
        return (best_threshold, best_gain). Implements zero‐division checks,
        min_sample_split enforcement, and the correct gain formula.
        """
        # Sort by feature value
        idx = np.argsort(feature_col)
        f = feature_col[idx]
        g_sorted = g[idx]
        h_sorted = h[idx]

        # Precompute totals
        G_total = g_sorted.sum()
        H_total = h_sorted.sum()
        parent_term = G_total**2 / (H_total + self.lamda)

        best_gain = -np.inf
        best_thr = None

        # Running sums for left node
        G_left = 0.0
        H_left = 0.0
        n = len(f)

        for i in range(n - 1):
            G_left += g_sorted[i]
            H_left += h_sorted[i]
            G_right = G_total - G_left
            H_right = H_total - H_left

            # Skip if feature values are identical (no split)
            if f[i] == f[i + 1]:
                continue

            # Enforce minimum samples in each split
            if (i + 1) < self.min_sample_split or (n - i - 1) < self.min_sample_split:
                continue

            # Compute denominators, skip if non‐positive
            den_left  = H_left  + self.lamda
            den_right = H_right + self.lamda
            if den_left <= 0 or den_right <= 0:
                continue

            # Gain = improvement over parent minus gamma
            gain = (G_left**2  / den_left +
                    G_right**2 / den_right -
                    parent_term) - self.gamma

            if gain > best_gain:
                best_gain = gain
                best_thr = (f[i] + f[i + 1]) / 2.0

        if best_thr is None:
            return None, 0.0
        return best_thr, best_gain


In [4]:
# TODO: class of Random Forest
class RF(object):
    '''
    Class of Random Forest
    
    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        loss: Loss function for gradient boosting.
            'mse' for regression task and 'log' for classfication task.
            A child class of the loss class could be passed to implement customized loss.
        max_depth: The maximum depth d_max of a tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf score, also known as lambda.
        gamma: The regularization coefficient for number of tree nodes, also know as gamma.
        rf: rf*m is the size of random subset of features, from which we select the best decision rule.
        num_trees: Number of trees.
    '''
    def __init__(self,
        n_threads = None, loss = 'mse',
        max_depth = 3, min_sample_split = 10, 
        lamda = 1, gamma = 0,
        rf = 0.99, num_trees = 100):
        
        self.n_threads = n_threads
        self.loss = loss
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.rf = rf
        self.num_trees = num_trees
        self.trees = []
        self.loss = leastsquare() if loss == 'mse' else logistic()
        self.is_classification = True if loss != 'mse' else False

    def fit(self, train, target):
        # train is n x m 2d numpy array
        # target is n-dim 1d array
        #TODO
        # Initialize trees
        self.trees = [Tree(n_threads=self.n_threads, 
                           max_depth=self.max_depth, 
                           min_sample_split=self.min_sample_split,
                           lamda=self.lamda, 
                           gamma=self.gamma, 
                           rf=self.rf) for _ in range(self.num_trees)]
        # Fit each tree
        for tree in self.trees:
            # Calculate gradients and hessians
            score = np.zeros(target.shape)
            idx = np.random.choice(np.arange(train.shape[0]), train.shape[0], replace=True)
            X_sample, y_sample = train[idx, :], target[idx]
            g, h = self.loss.g(y_sample, score), self.loss.h(y_sample, score)
            tree.fit(X_sample, g, h)
        return self

    def predict(self, X):
        raw = np.vstack([tree.predict(X) for tree in self.trees])
        if self.is_classification:
            score = raw.mean(axis=0)
            return self.loss.pred(score)
        else:
            return raw.mean(axis=0)

In [5]:
# TODO: Evaluation functions (you can use code from previous homeworks)

# RMSE
def root_mean_square_error(pred, y):
    #TODO
    n = pred.shape[0]
    return np.sqrt(np.sum((pred - y) ** 2) / n)
    

# precision
def accuracy(pred, y):
    #TODO
    n = pred.shape[0]
    return np.sum(pred == y) / n

In [6]:
# TODO: class of GBDT
class GBDT(object):
    '''
    Class of gradient boosting decision tree (GBDT)
    
    Parameters:
        n_threads: The number of threads used for fitting and predicting.
        loss: Loss function for gradient boosting.
            'mse' for regression task and 'log' for classfication task.
            A child class of the loss class could be passed to implement customized loss.
        max_depth: The maximum depth D_max of a tree.
        min_sample_split: The minimum number of samples required to further split a node.
        lamda: The regularization coefficient for leaf score, also known as lambda.
        gamma: The regularization coefficient for number of tree nodes, also know as gamma.
        learning_rate: The learning rate eta of GBDT.
        num_trees: Number of trees.
    '''
    def __init__(self,
        n_threads = None, loss = 'mse',
        max_depth = 3, min_sample_split = 10, 
        lamda = 1, gamma = 0,
        learning_rate = 0.1, num_trees = 100):
        
        self.n_threads = n_threads
        self.loss = leastsquare() if loss == 'mse' else logistic()
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
        self.lamda = lamda
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.num_trees = num_trees
        self.is_classification = True if loss != 'mse' else False

    def fit(self, train, target):
        # train is n x m 2d numpy array
        # target is n-dim 1d array
        #TODO
        n_samples, n_features = train.shape
        self.trees = []
        self.train = train
        self.target = target
        self.score = np.zeros(n_samples)
        for _ in range(self.num_trees):
            # Calculate gradients and hessians
            g = self.loss.g(target, self.score)
            h = self.loss.h(target, self.score)
            # Fit a tree to the gradients and hessians
            tree = Tree(n_threads=self.n_threads, 
                        max_depth=self.max_depth, 
                        min_sample_split=self.min_sample_split,
                        lamda=self.lamda, 
                        gamma=self.gamma, rf=0)
            tree.fit(train, g, h)
            # Update the score with the predictions from the tree
            self.score += self.learning_rate * tree.predict(train)
            # Store the tree
            self.trees.append(tree)
        return self
    
    def predict(self, test):
        score = np.zeros(test.shape[0])
        if self.is_classification:
            for tree in self.trees:
                score += self.learning_rate * tree.predict(test)
            # 如果 loss.pred 只做 sigmoid → 0~1 概率：
            proba = self.loss.pred(score)
            return (proba > 0.5).astype(int)
        else:
            for tree in self.trees:
                score += self.learning_rate * tree.predict(test)
            return score

In [7]:
# TODO: GBDT regression on boston house price dataset

# load data
import numpy as np
import pandas as pd

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]


# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
/var/folders/1l/_31fxbps5_db7ymc5ghngxz00000gn/T/ipykernel_72235/289120647.py:8: SyntaxWarning: invalid escape sequence '\s'
  raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)


(506, 13) (506,) (354, 13) (354,) (152, 13) (152,)


In [8]:
# fit housing price with Random Forest
model = RF(num_trees = 100, rf = 0.5, max_depth=10, min_sample_split=5)
model.fit(X_train, y_train)

pred_train = model.predict(X_train)
rmse_train = root_mean_square_error(pred_train, y_train)

pred_test = model.predict(X_test)
rmse_test = root_mean_square_error(pred_test, y_test)

print('train rmse: {} | test rmse: {}'.format(rmse_train, rmse_test))

train rmse: 3.172882841926914 | test rmse: 3.8463298122461476


In [9]:
# fit housing price with GBDT
model_gbdt = GBDT(
    n_threads= 1,
    loss='mse',
    max_depth=5,
    min_sample_split=10,
    lamda=1,
    gamma=0,
    learning_rate=0.1,
    num_trees=50,
)

model_gbdt.fit(X_train, y_train)
pred_train_gbdt = model_gbdt.predict(X_train)
rmse_train_gbdt = root_mean_square_error(pred_train_gbdt, y_train)
pred_test_gbdt = model_gbdt.predict(X_test)
rmse_test_gbdt = root_mean_square_error(pred_test_gbdt, y_test)
print('GBDT train rmse: {} | GBDT test rmse: {}'.format(rmse_train_gbdt, rmse_test_gbdt))

GBDT train rmse: 1.5963580003353244 | GBDT test rmse: 3.587045129457542


In [10]:
# fit housing price with linear regression
W = np.matmul(
    np.linalg.inv(np.matmul(X_train.T, X_train)),
    np.matmul(X_train.T, y_train))


pred_train = np.matmul(X_train, W)
rmse_train = root_mean_square_error(pred_train, y_train)

pred_test = np.matmul(X_test, W)
rmse_test = root_mean_square_error(pred_test, y_test)

print('train rmse: {} | test rmse: {}'.format(rmse_train, rmse_test))

train rmse: 4.820626531838222 | test rmse: 5.209217510530889


In [11]:
# fit housing price with ridge regression 
from sklearn.linear_model import Ridge
ridge_model = Ridge(alpha=0.5)
ridge_model.fit(X_train, y_train)
pred_train = ridge_model.predict(X_train)
rmse_train = root_mean_square_error(pred_train, y_train)
pred_test = ridge_model.predict(X_test)
rmse_test = root_mean_square_error(pred_test, y_test)
print('train rmse: {} | test rmse: {}'.format(rmse_train, rmse_test))

train rmse: 4.636494716898294 | test rmse: 4.9243576920779315


In [12]:
# TODO: GBDT classification on credit-g dataset

# load data
from sklearn.datasets import fetch_openml
X, y = fetch_openml('credit-g', version=1, return_X_y=True, data_home='credit/')
y = np.array(list(map(lambda x: 1 if x == 'good' else 0, y)))


X = pd.get_dummies(X)
X = X.values


# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# fit credit-g with GBDT 
model_gbdt = GBDT(
    n_threads=8,
    loss='log',
    max_depth=4,
    min_sample_split=10,
    lamda=1,
    gamma=0,
    learning_rate=0.1,
    num_trees=50
)
model_gbdt.fit(X_train, y_train)
pred_train_gbdt = model_gbdt.predict(X_train)
acc_train_gbdt = accuracy(pred_train_gbdt, y_train)
pred_test_gbdt = model_gbdt.predict(X_test)
acc_test_gbdt = accuracy(pred_test_gbdt, y_test)
print('GBDT train acc: {} | GBDT test acc: {}'.format(acc_train_gbdt, acc_test_gbdt))

(1000, 61) (1000,) (700, 61) (700,) (300, 61) (300,)
GBDT train acc: 0.8914285714285715 | GBDT test acc: 0.7633333333333333


In [13]:
# RF on credit-g
rf_model = RF(
    loss=logistic(),
    max_depth=5,
    min_sample_split=5,
    lamda=1.5,
    gamma=0,
    rf=0.7,
    num_trees=100
)

rf_model.fit(X_train, y_train)

y_pred_train_prob = rf_model.predict(X_train)
y_pred_test_prob = rf_model.predict(X_test)

y_pred_train = (y_pred_train_prob > 0.5).astype(int)
y_pred_test = (y_pred_test_prob > 0.5).astype(int)

train_accuracy = accuracy(y_pred_train, y_train)
test_accuracy = accuracy(y_pred_test, y_test)

print(f"Random Forest Training Accuracy (Credit-g): {train_accuracy}")
print(f"Random Forest Test Accuracy (Credit-g): {test_accuracy}")

Random Forest Training Accuracy (Credit-g): 0.8385714285714285
Random Forest Test Accuracy (Credit-g): 0.7466666666666667


In [14]:
# TODO: GBDT classification on breast cancer dataset

# load data
from sklearn import datasets
breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)
print(X.shape, y.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# X = pd.get_dummies(X)
# X = X.values
# fit breast cancer with GBDT
model_gbdt = GBDT(
    n_threads=8,
    loss='log',
    max_depth=3,
    min_sample_split=10,
    lamda=1,
    gamma=0,
    learning_rate=0.1,
    num_trees=50
)
model_gbdt.fit(X_train, y_train)
pred_train_gbdt = model_gbdt.predict(X_train)
acc_train_gbdt = accuracy(pred_train_gbdt, y_train)
pred_test_gbdt = model_gbdt.predict(X_test)
acc_test_gbdt = accuracy(pred_test_gbdt, y_test)
print('GBDT train acc: {} | GBDT test acc: {}'.format(acc_train_gbdt, acc_test_gbdt))


(569, 30) (569,) (398, 30) (398,) (171, 30) (171,)
GBDT train acc: 0.9974874371859297 | GBDT test acc: 0.9590643274853801


In [15]:
# classify breast cancer with RF
rf_model = RF(
    loss=logistic(),
    max_depth=10,
    min_sample_split=5,
    lamda=2,
    gamma=1,
    rf=0.7,
    num_trees=100,
)

rf_model.fit(X_train, y_train)

y_pred_train_prob = rf_model.predict(X_train)
y_pred_test_prob = rf_model.predict(X_test)

y_pred_train = (y_pred_train_prob > 0.5).astype(int)
y_pred_test = (y_pred_test_prob > 0.5).astype(int)

train_accuracy = accuracy(y_pred_train, y_train)
test_accuracy = accuracy(y_pred_test, y_test)

print(f"Random Forest Training Accuracy (breast cancer): {train_accuracy}")
print(f"Random Forest Test Accuracy (breast cancer): {test_accuracy}")

Random Forest Training Accuracy (breast cancer): 0.9798994974874372
Random Forest Test Accuracy (breast cancer): 0.9707602339181286
